In [3]:
import os

import numpy as np
import pandas as pd

In [4]:
root_paths = [
    "/data/kaggle-wikipedia/data2/",
    "/Users/jiayou/Dropbox/JuanCode/Insight/Project/tf-data/",
    "/Users/jiayou/Dropbox/Documents/JuanCode/Kaggle/Wikipedia/data2/",
    '/mnt/WD Black/Dropbox/JuanCode/Insight/Project/tf-data/',
    '/home/paperspace/Documents/insight/insight/tf-data/',
]
root = None
for p in root_paths:
    if os.path.exists(p):
        root = p
        break
print(root)

/home/paperspace/Documents/insight/insight/tf-data/


In [22]:
preds_mat = np.load(os.path.join(root, 'predictions', 'preds.npy'))
labels_mat = np.load(os.path.join(root, 'predictions', 'labels.npy'))
priors_mat = np.load(os.path.join(root, 'predictions', 'priors.npy'))
pages_mat = np.load(os.path.join(root, 'predictions', 'page_id.npy'))

In [23]:
df = pd.read_csv('data2/train_2.csv', encoding='utf-8')
date_cols = [i for i in df.columns if i != 'Page']
datetimes = pd.to_datetime(date_cols, format="%Y/%m/%d")

In [24]:
next_date_cols = pd.date_range(start=datetimes[-1], periods=65, closed='right')

In [26]:
pred_df = pd.DataFrame(preds_mat, columns=next_date_cols)
pred_df['page_id'] = pages_mat

page_id_df = pd.read_csv(root+'processed_full/page_ids.csv', encoding='utf-8')
pred_df = pred_df.merge(page_id_df, how='left', on='page_id')

submit_cols = list(next_date_cols[2:]) + ['Page']
pred_df = pred_df[submit_cols]
pred_df = pd.melt(pred_df, id_vars='Page', var_name='date', value_name='Visits')

keys = pd.read_csv('data2/key_2.csv', encoding='utf-8')
keys['date'] = keys.Page.apply(lambda a: a[-10:])
keys['Page'] = keys.Page.apply(lambda a: a[:-11])
keys['date'] = keys['date'].astype('datetime64[ns]')

pred_df = pred_df.merge(keys, how='left', on=['Page', 'date'])
pred_df['Visits'] = pred_df['Visits'].map(np.round).astype(int)
pred_df = pred_df[['Id', 'Visits']].sort_values(by='Id')

IsADirectoryError: [Errno 21] Is a directory: '/home/paperspace/Documents/insight/insight/tf-data/'

In [28]:
pred_df.to_csv(root+'sub.csv', encoding='utf-8', index=False)